In [ ]:
import random
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Загрузим пример данных
data = load_iris()
X = data.data
y = data.target

# Разделим на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Функция для генерации случайных параметров для CatBoost
def random_catboost_params():
    return {
        'iterations': random.choice([100, 200, 300]),
        'depth': random.choice([4, 6, 10]),
        'learning_rate': random.choice([0.01, 0.1, 0.2]),
    }

# Создание списка моделей CatBoost с рандомными параметрами
models = []
for _ in range(5):  # например, 5 моделей CatBoost
    params = random_catboost_params()
    model = CatBoostClassifier(**params, verbose=0)
    models.append(('catboost', model))

# Создадим VotingClassifier
voting_clf = VotingClassifier(estimators=models, voting='soft')

# Обучение модели
voting_clf.fit(X_train, y_train)

# Прогнозирование
y_pred = voting_clf.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


In [ ]:
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Загрузим пример данных
data = load_iris()
X = data.data
y = data.target

# Разделим на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Параметры для разных моделей CatBoost
params_list = [
    {'iterations': 100, 'depth': 4, 'learning_rate': 0.05, 'loss_function': 'MultiClass', 'verbose': 0},
    {'iterations': 150, 'depth': 6, 'learning_rate': 0.1, 'loss_function': 'MultiClass', 'verbose': 0},
    {'iterations': 200, 'depth': 8, 'learning_rate': 0.1, 'loss_function': 'MultiClass', 'verbose': 0},
    {'iterations': 100, 'depth': 5, 'learning_rate': 0.2, 'loss_function': 'MultiClass', 'verbose': 0},
    {'iterations': 250, 'depth': 7, 'learning_rate': 0.05, 'loss_function': 'MultiClass', 'verbose': 0}
]

# Создание списка моделей CatBoost с разными параметрами
models = []
for i, params in enumerate(params_list):
    model = CatBoostClassifier(**params)
    models.append((f'catboost_{i}', model))

# Создадим VotingClassifier
voting_clf = VotingClassifier(estimators=models, voting='soft')

# Обучение модели
voting_clf.fit(X_train, y_train)

# Прогнозирование
y_pred = voting_clf.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_csv('train.csv')  # Замените на ваш путь к данным

# Предположим, что у вас есть признаки в колонках X и целевая переменная в y
X = data.drop(columns=['target'])  # Замените на ваш набор признаков
y = data['target']  # Замените на вашу целевую переменную

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание данных для LGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Параметры модели
params = {
    'objective': 'binary',  # для бинарной классификации, для многоклассов можно использовать 'multiclass'
    'metric': 'binary_error',  # метрика для бинарной классификации
    'boosting_type': 'gbdt',  # тип бустинга
    'num_leaves': 31,  # количество листьев
    'learning_rate': 0.05,  # скорость обучения
    'feature_fraction': 0.9  # случайный выбор признаков на каждом шаге
}

# Обучение модели
bst = lgb.train(params, train_data, valid_sets=[test_data], early_stopping_rounds=10)

# Прогнозирование
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Преобразование вероятностей в метки классов
y_pred_label = (y_pred > 0.5).astype(int)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred_label)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Загрузка данных
data = pd.read_csv('train.csv')  # Замените на ваш путь к данным

# Предположим, что у вас есть признаки в колонках X и целевая переменная в y
X = data.drop(columns=['target'])  # Замените на ваш набор признаков
y = data['target']  # Замените на вашу целевую переменную

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразование данных в формат DMatrix для XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Параметры модели
params = {
    'objective': 'binary:logistic',  # для бинарной классификации, для многоклассов используйте 'multi:softmax'
    'eval_metric': 'logloss',  # метрика для бинарной классификации
    'max_depth': 6,  # максимальная глубина дерева
    'learning_rate': 0.05,  # скорость обучения
    'subsample': 0.8,  # доля выборки для построения дерева
    'colsample_bytree': 0.8  # доля признаков для каждого дерева
}

# Обучение модели
num_round = 100  # количество итераций
bst = xgb.train(params, dtrain, num_round, [(dtest, 'eval')], early_stopping_rounds=10)

# Прогнозирование
y_pred = bst.predict(dtest)

# Преобразование вероятностей в метки классов
y_pred_label = (y_pred > 0.5).astype(int)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred_label)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
import catboost
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRanker, Pool

# Загрузка данных
data = pd.read_csv('train.csv')  # Замените на ваш путь к данным

# Предположим, что у вас есть признаки в колонках X и целевая переменная в y
X = data.drop(columns=['target', 'group_id'])  # Замените на ваш набор признаков
y = data['target']  # Замените на вашу целевую переменную
group = data['group_id']  # Предполагаем, что у вас есть колонка group_id для группы документов

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(X, y, group, test_size=0.2, random_state=42)

# Создание Pool данных для CatBoost
train_data = Pool(X_train, label=y_train, group_id=group_train)
test_data = Pool(X_test, label=y_test, group_id=group_test)

# Параметры модели CatBoost
model = CatBoostRanker(iterations=1000, 
                       depth=6, 
                       learning_rate=0.05, 
                       loss_function='RMSE', 
                       custom_metric=['NDCG'], 
                       cat_features=[])

# Обучение модели
model.fit(train_data, eval_set=test_data, early_stopping_rounds=10)

# Прогнозирование
y_pred = model.predict(X_test)

# Оценка качества модели
# Например, использование метрики NDCG для оценки качества ранга
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.4f}')
